In [1]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from utils import *

import pandas as pd
import numpy as np
from glob import glob
from keras.layers import Dense, Conv2D, BatchNormalization, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.metrics import log_loss
from PIL import Image
%matplotlib inline

cwd = os.getcwd()
TARGET_SIZE= (224, 224)
BSIZE= 32
input_dir = os.path.join(os.path.dirname(cwd), 'data', 'crops_for_rot', 'wrap')


Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


# Compute Features

In [2]:
def get_b(dr = input_dir, gen = ImageDataGenerator(), batch_size=BSIZE):
    return gen.flow_from_directory(dr, shuffle=False, target_size=TARGET_SIZE, batch_size=BSIZE)


In [3]:
tr_b = get_b(input_dir +'/train')

Found 3864 images belonging to 447 classes.


In [4]:
import vgg16bn
vgg = Vgg16BN(size=TARGET_SIZE, include_top=False)
model = vgg.model
for l  in model.layers: l.trainable = False

In [5]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  0           zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [6]:
tr_feats = model.predict_generator(tr_b, val_samples=tr_b.nb_sample)

In [7]:
tr_feats.shape

(3864, 512, 7, 7)

In [8]:
% cd $cwd

/home/ubuntu/courses/deeplearning1/nbs/whale_recognition


In [9]:
save_array('tr_rot_224_conv_feats.bc', tr_feats)

In [10]:
val_b = get_b(input_dir +'/valid')

Found 680 images belonging to 447 classes.


In [11]:
val_feats = model.predict_generator(val_b, val_samples=val_b.nb_sample)

In [12]:
val_feats.shape

(680, 512, 7, 7)

In [13]:
save_array('val_rot_224_conv_feats.bc', val_feats)

In [17]:
test_b = get_b(input_dir +'/test')

Found 6925 images belonging to 1 classes.


In [18]:
test_feats = model.predict_generator(test_b, val_samples=test_b.nb_sample)

In [19]:
test_feats.shape

(6925, 512, 15, 15)

In [21]:
save_array('test_rot_500_conv_feats.bc', test_feats)